In [27]:
from DCNN import DCNN
import os
import librosa
import numpy as np

# Read Data

In [28]:
root = 'Data/genres_original'
genres = os.listdir(root)
x = []
y = []
length = []
sr = 22050
for genre in genres:
    genre_root = os.path.join(root, genre)
    audios = os.listdir(genre_root)
    for audio in audios:
        audio_path = os.path.join(genre_root, audio)
        signal, _ = librosa.load(audio_path, sr=sr)
        x.append(signal)
        length.append(len(signal))
        y.append(genre)
min_length = min(length)

# Segment and Normalise 

In [30]:
for i in range(1000):
    x[i] = x[i][0:min_length]
    x[i] = librosa.util.normalize(x[i])
x = np.asarray(x)
y = np.asarray(y)
print(x.shape,y.shape)

(1000, 660000) (1000,)
